In [32]:
import numpy as np
import pandas as pd
import random
import tqdm
from collections import deque
from tqdm import tqdm

In [76]:
#set environment
def grid(size):
    ground = np.zeros((size,size)) 
    x=random.sample(range(0, size), 3)
    y=random.sample(range(0, size), 3)
    
    ground[x[0],y[0]]=1
    ground[x[1],y[1]]=1
    ground[x[2],y[2]]=2
    
    return ground,x,y
class curr_config:
    def __init__(self,sspos,d1pos,d2pos,size):
        
        self.s = (sspos//size,sspos%size)
        self.d1 = (d1pos//size,d1pos%size)
        self.d2 = (d2pos//size,d2pos%size)
        self.u_value = 0
        self.r_flag=False

In [77]:
#define potential states
def potential_state(state_space,state_pos,poss_action,size):
    p_state = [] 

    d1_pos = state_space[state_pos]['dog1']; d2_pos = state_space[state_pos]['dog2']; s_pos = state_space[state_pos]['sheep']
    new_d1_pos = (d1_pos[0]+poss_action[0][0],d1_pos[1]+poss_action[0][1])
    new_d2_pos = (d2_pos[0]+poss_action[1][0],d2_pos[1]+poss_action[1][1])
    s_possible_move=[(1,0),(-1,0),(0,1),(0,-1)]
    for i in range(4):
        new_s_pos = (s_pos[0]+s_possible_move[i][0],s_pos[1]+s_possible_move[i][1])
        if(edge_case(new_s_pos,size) and new_s_pos!=new_d1_pos and new_s_pos!=new_d2_pos ): p_state.append(state_space[str(new_s_pos)+str(new_d1_pos)+str(new_d2_pos)])
    
    if(len(p_state)==0):
        p_state.append(state_space[str(s_pos)+str(new_d1_pos)+str(new_d2_pos)])
    return p_state

In [78]:
# Considering edge cases
def edge_case(curr_pos,size):
    if(curr_pos[0]>=0 and curr_pos[0]<=size-1 and curr_pos[1]>=0 and curr_pos[1]<=size-1):
        return True
    return False
    

In [79]:
# all possible states
def all_possible_states(size):
    number_cells = size*size
    config = number_cells * number_cells * number_cells
    class_obj_state=[0 for i in range(config)]
    state_space={}
    for i in range(number_cells):
        for j in range(number_cells):
            if(j!=i):
                for k in range(number_cells):
                    if(k!=i and k!=j):
                        class_obj_state[i*number_cells*number_cells + j*number_cells + k]=curr_config(i,j,k,size)
                        sl=class_obj_state[i*number_cells*number_cells + j*number_cells + k]
                        state_space[str(sl.s)+str(sl.d1)+str(sl.d2)]={'sheep':sl.s,'dog1':sl.d1,'dog2':sl.d2,'utility_value':sl.u_value,'reward_flag':sl.r_flag}
    return state_space
                        

In [80]:
# getting atction state sapce
def action_state(state_space,state_pos,size):
    
    d1_possible_move=[(1,0),(-1,0),(0,1),(0,-1)]
    d2_possible_move=[(1,0),(-1,0),(0,1),(0,-1)]
    poss_actions=[]
    d1_pos = state_space[state_pos]['dog1']; d2_pos = state_space[state_pos]['dog2']; new_s_pos = state_space[state_pos]['sheep']

    for i in range(4):
        for j in range(4):
            new_d1_pos = (d1_pos[0]+d1_possible_move[i][0],d1_pos[1]+d1_possible_move[i][1])
            new_d2_pos = (d2_pos[0]+d2_possible_move[j][0],d2_pos[1]+d2_possible_move[j][1])
            if(edge_case(new_d1_pos,size) and edge_case(new_d2_pos,size) and new_d1_pos!=new_d2_pos and  new_s_pos!=new_d1_pos and new_s_pos!=new_d2_pos):
                poss_actions.append((d1_possible_move[i],d2_possible_move[j]))
    return poss_actions

In [81]:
# defining rewards
def calcreward(state_space,state_pos,poss_action,size):
    
    s_pos = state_space[state_pos]['sheep'];d1_pos = state_space[state_pos]['dog1'];d2_pos = state_space[state_pos]['dog2']
    if(s_pos == (size-1,size-1) and ((d1_pos ==(size-1,size-2) and d2_pos ==(size-2,size-1)) or (d1_pos ==(size-2,size-1) and d2_pos ==(size-1,size-2)) ) ):
        return 0
    if(state_space[state_pos]['reward_flag']==True):
        return 1
    else:
        return -1
    

In [82]:
#cal current utilities
def curr_utility(state_pos,poss_action,size,state_space):
    
    new_s_pos = (size-1,size-1)
    new_d1_pos = (size-2,size-1)
    new_d2_pos = (size-2,size-1)    
    goal_state1 = str(new_s_pos)+str(new_d1_pos)+str(new_d2_pos)
    goal_state2 = str(new_s_pos)+str(new_d2_pos)+str(new_d1_pos)
    p_state = potential_state(state_space,state_pos,poss_action,size)
    
    if(state_space[state_pos] == goal_state1 or state_space[state_pos] == goal_state2):
        return 1
    if(goal_state1 in p_state or goal_state2 in p_state ):
        return 1
    
    tot_p_state = len(p_state)
    prob = 0 
    if(tot_p_state == 0):
        util =  state_space[state_pos]['utility_value']
        return util
    else:
        util=0
        prob = 1/tot_p_state
        for i in range(len(p_state)):
            util+=prob*p_state[i]['utility_value']
        return util

In [83]:
#updating utilities
def update_util(state_space,state_pos,size):
    new_s_pos = (size-1,size-1)
    new_d1_pos = (size-2,size-1)
    new_d2_pos = (size-2,size-1)
    
    goal_state1 = str(new_s_pos)+str(new_d1_pos)+str(new_d2_pos)
    goal_state2 = str(new_s_pos)+str(new_d2_pos)+str(new_d1_pos)
    
    state=state_space[state_pos]
    if(state_space[state_pos] == goal_state1 or state_space[state_pos] == goal_state2):
        return 0
    poss_actions=action_state(state_space,state_pos,size)
    utili_v=[0 for i in range(len(poss_actions))]
    maximum_u=-100000000
    for i in range(len(poss_actions)):
        reward=calcreward(state_space,state_pos,poss_actions[i],size)
        expected = curr_utility(state_pos,poss_actions[i],size,state_space)
        if(expected==0):
            state_space[state_pos]['reward_flag']=True
            return reward +expected
        utili_v[i]=reward +expected
        if(utili_v[i]>maximum_u):
            maximum_u=utili_v[i]
    return maximum_u

In [87]:
#run till convergence
def sdbot(size):
    states = all_possible_states(size)
    for m in tqdm(range(1000)):
        new_utility=[0 for i in range(len(states))]
        c=0
        for k,v in states.items():            
                new_utility[c] = update_util(states,k,size)
                c=c+1
        l=0
        for k,v in states.items():
            states[k]['utility_value']=new_utility[l]
            l=l+1
    return states

In [92]:
size=7 #7x7 grid
Final_states = sdbot(size) 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 29.72it/s]


In [93]:

for val in Final_states:

    print(Final_states[val]['sheep'],Final_states[val]['dog1'],Final_states[k]['dog2'],Final_states[val]['utility_value'])
    print(abs(Final_states[val]['utility_value']))

(0, 0) (0, 1) (0, 2) 98.0
98.0
(0, 0) (0, 1) (0, 2) 97.99999999999994
97.99999999999994
(0, 0) (0, 1) (0, 2) 98.0
98.0
(0, 0) (0, 1) (0, 2) 97.99999999999994
97.99999999999994
(0, 0) (0, 1) (0, 2) 98.0
98.0
(0, 0) (0, 1) (0, 2) 97.99999999999994
97.99999999999994
(0, 0) (0, 1) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (0, 2) (0, 2) 98.0
98.0
(0, 0) (1, 0) (0, 2) 97.99999999999994
97.99999999999994
(0, 0) (1, 0) (0, 2) 98.0
98.0
(0, 0) (1, 0) (0, 2) 98.0
98.0
(0, 0) (1, 0) (0, 2) 97.99999999999994
97.99999999999994
(0, 0) (1, 0) (0, 2) 98.0
98.0
(0, 0) (1, 0) (0, 2) 97.99999999999994
97.99999999999994
(0, 0) (1, 0) (0, 2) 98.0
98.0
(0, 0) (1, 1) (0, 2) 98.0
98.0
(0, 0) (1, 1) (0, 2) 98.0
98.0
(0, 0) (1, 1) (0, 2) 98.0
98.0
(0, 0) (1, 1) (0, 2) 98.0
98.0
(0, 0) (1, 1) (0, 2) 98.0
98.0
(0, 0) (1, 1) (0, 2) 98.0
98.0
(0, 0) 

98.0
(2, 1) (0, 0) (0, 2) 98.0
98.0
(2, 1) (0, 0) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 0) (0, 2) 98.0
98.0
(2, 1) (0, 0) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 0) (0, 2) 98.0
98.0
(2, 1) (0, 0) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 0) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 1) (0, 2) 98.0
98.0
(2, 1) (0, 2) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 2) (0, 2) 98.0
98.0
(2, 1) (0, 2) (0, 2) 98.0
98.0
(2, 1) (0, 2) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 2) (0, 2) 98.0
98.0
(2, 1) (0, 2) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (0, 2) (0, 2) 97.99999999999994
97.99999999999994
(2, 1) (1, 0) (0, 2) 98.0
98.0
(2, 1) (1, 0) (0, 2) 98.0
98.0
(2, 1) (1, 0) (0, 2) 98.0
98.0
(2, 1) (1, 0) (0, 2) 98.0
98.0
(2, 1) (1, 0

1 4
